In [7]:
# 前提：横書き日本語（一部英語を含む）で書かれた本を1ページごとにPNGファイルとして保存してある
#　PNGファイルはscreenshotFolderに入っている。
#　PNGファイルをトリミングして不要な余白を削除し、inputフォルダに入れる。
#　PNGファイルを個々にOCR処理し、その結果をPDF（PNG画像に透明テキストを貼り付けたもの）に変換する。
#　PDFファイルはoutputフォルダに格納される。
#　outputフォルダ内のPDFファイルを統合してPrivateEBooksフォルダに保存する。

from PIL import Image
import os
from PyPDF4 import PdfFileReader, PdfFileWriter
import tkinter as tk
from tkinter import simpledialog
                     
#　指定したフォルダ内にあるPNGファイルをトリミングし、別に指定したフォルダに保存する。
def trimming(folder_path, save_path, left, upper, right, lower):
    #  folder_path:元画像（PNG）が入っているフォルダへのパス
    #  save_path：トリミング後のファイルを保存するフォルダへのパス
    #　left, upper, right, lowerトリミングするピクセル座標
    # トリミングするPNGファイルへのパスのリスト
    png_files = [f for f in os.listdir(folder_path) if f.endswith('.png')]
    for file in png_files:
        # 画像を開く
        im = Image.open(folder_path + '\\'+ file)
        # 座標で切り抜く
        im_crop = im.crop((left, upper, right, lower))
        # 新しいファイル名で保存
        im_crop.save(save_path + '\\'+ file)

def merge_pdfs(output_folder, output_filename, save_folder):
    pdf_writer = PdfFileWriter()

    # outputフォルダ内のPDFファイルを読み込んで結合
    for file in os.listdir(output_folder):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(output_folder, file)
            pdf_reader = PdfFileReader(pdf_path)

            for page_num in range(pdf_reader.getNumPages()):
                pdf_writer.addPage(pdf_reader.getPage(page_num))

    # 結合したPDFを単一ファイルとして保存
    with open(os.path.join(save_folder, output_filename), "wb") as output_file:
        pdf_writer.write(output_file)



root = tk.Tk()#ルートウィンドウを作成する
root.withdraw()#ルートウィンドウを隠す

BookTitle = simpledialog.askstring(title="本のタイトル",
                                  prompt="書籍の名前を入力してください:")

#　トリミング処理
# トリミングしたい座標
left = 118
upper = 100
right = 900
lower = 1000

# トリミングするPNGファイルが入ったフォルダのパス
folder_path = r"C:\Users\shin\Desktop\screenshotFolder"

# トリミングした後のPNGファイルを入れるフォルダのパス
save_path = r"C:\Users\shin\Documents\PrivateEBooks\input"

# トリミング実行
trimming(folder_path, save_path, left, upper, right, lower)

# OCR処理を行う
# CDR処理用ps1ファイルのパスを変数に代入
ps1_file = r"C:\Users\shin\Documents\PrivateEBooks\OCR_ShellScript.ps1"

# PowerShellを起動してps1ファイルを実行するコマンドを変数に代入
command = "powershell.exe -ExecutionPolicy Bypass -File " + ps1_file

# os.system関数でコマンドを実行
os.system(command)

#　PDFファイルをまとめて一つのファイルにする
#　個別ページのPDFが入っているフォルダへのパス
output_folder = "output"

#　結合したPDFファイルを保存するフォルダのパス
Ebook_folder = r"C:\Users\shin\Documents\PrivateEBooks\PDFBooks"

merge_pdfs(output_folder, BookTitle + '.pdf', Ebook_folder)

#　途中経過ファイルの削除
delete_files_in_folder(folder_path)
delete_files_in_folder(input_folder)
delete_files_in_folder(output_folder)

: 